# Implementación y Ejecución de Algortimo de Prueba
<br/>
Se modificó el código visto en clases para encender el LED verde para el host maestro y la rojo para los esclavos. El algoritmo funciona de la siguiente manera:
<br/>
Si el host es maestro(rank=0) envía un mensaje a todos los demás y comienza a recibir las respuesta de cada uno de los esclavos.
Para cada respuesta enciende el ping correspondiente al esclavo de forma predefinida.
<br/>
Cuando termina de recibir todas las respuestas esperadas de cada proceso envia una señal (-1) de apagado y limpia los GPIO apagándolos todos.

 


In [ ]:
"""
Prueba de encendido de LEDs 
"""

from mpi4py import MPI
import RPi.GPIO as GPIO
import time as time
# Set up GPIO
GPIO.cleanup()
GPIO.setmode(GPIO.BCM)
GPIO.setup(27, GPIO.OUT) #Mestro Proceso 0, led verde
GPIO.setup(18, GPIO.OUT) #Proceso 1, led rojo
GPIO.setup(23, GPIO.OUT) #Proceso 2, led rojo
GPIO.setup(24, GPIO.OUT) #Proceso 3, led rojo
GPIO.setup(25, GPIO.OUT) #Proceso 4, led rojo

# MPI variables for communication, rank, size, and name of the node
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
name = MPI.Get_processor_name()

total_requests = 12
print "rank=",rank
# This is the master node.
if rank == 0:
    request = 0
    process = 1

      # Send the first batch of processes to the nodes.
    while process < size and request < total_requests:
        comm.send(request, dest=process, tag=1)
        print "Sending request",request,"to process",process
        request += 1
        process += 1

    # Wait for the data to come back
    received_processes = 0
    while received_processes < total_requests:
        process = comm.recv(source=MPI.ANY_SOURCE, tag=1)
        print "Received data from process", process
        if (process == 1):
            GPIO.output(18,True)
            time.sleep(1)
            GPIO.output(18,False)
            GPIO.output(23, False)
            GPIO.output(24, False)
            GPIO.output(25, False)

        elif (process == 2):
            GPIO.output(23, True)
            time.sleep(1)
            GPIO.output(23,False)
            GPIO.output(18, False)
            GPIO.output(24, False)
            GPIO.output(25, False)
        elif (process == 3):

            GPIO.output(24, True)
            time.sleep(1)
            GPIO.output(24,False)
            GPIO.output(18, False)
            GPIO.output(23, False)
            GPIO.output(25, False)
        elif (process == 4):
            GPIO.output(25, True)
            time.sleep(1)
            GPIO.output(25,False)
            GPIO.output(18, False)
            GPIO.output(23, False)
            GPIO.output(24, False)

        received_processes += 1

        if request < total_requests:
            comm.send(request, dest=process, tag=1)
            print "Sending request",request,"to process", process
            GPIO.output(27, True)
            request += 1
# Send the shutdown signal
    for process in range(1,size):
        comm.send(-1, dest=process, tag=1)
    print "End"


# These are the slave nodes, where rank > 0. They receive the request from the master node and simply send a response back
else:
    while True:
        start = comm.recv(source=0, tag=1)
        if start == -1: break
        comm.send(rank, dest=0, tag=1)


#Clean up GPIO activities after all processing takes place
GPIO.cleanup()

# Procedimiento del Examen
Se establece la conexión SSH con la Raspberry mediante el cliente Putty:
<br/>
<br/>
<img style="margin:0px auto;display:block" src="Putyy.png"/>


 Una vez que se introduce el usuario (pi) y contraseña nos dirigimos a la carpeta donde se encuentra el script.
  <div style="background-color:black; color:green;widht:700px;height:50px">cd comade/python </div>
  <br/>
  Se modifica el archivo machinefile dejando solo la raspberry esclava (en este caso la 2) y nuestra raspberry.
  Ejecutamos el <b>mpirun</b> para dos procesos, cargando el machine file y el script de python 
  <br/>

 <div style="background-color:black; color:green;widht:700px;height:50px">mpirun.openmpi -np 2 -machinefile machinefile python ledequipo3.py <div/>